# Getting started with SQD

In this page we will overview the `qiskit-addons-sqd` package and demonstrate a simple working example to get started. We will demonstrate this tool by finding an approximation to the ground state of a 22-site XX-Z spin-1/2 chain. This example will use the `qiskit-addon-utils` package to assist in the problem generation.

To begin, recall that the Hamiltonian for this system has the form

$$ H = \sum_{\langle i, j \rangle} J_{xy}\left(X_iX_j + Y_iY_J \right) + Z_iZ_j. $$

Next the Hamiltonian of a 22-site chain can be generated by running

In [3]:
from qiskit.transpiler import CouplingMap
from qiskit_addon_utils.problem_generators import generate_xyz_hamiltonian

num_spins = 22
coupling_map = CouplingMap.from_ring(num_spins)
xxz_hamiltonian = generate_xyz_hamiltonian(coupling_map, coupling_constants=(0.3, 0.3, 1.0))
print(xxz_hamiltonian)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIXX', 'IIIIIIIIIIIIIIIIIIIIYY', 'IIIIIIIIIIIIIIIIIIIIZZ', 'IIIIIIIIIIIIIIIIIIXXII', 'IIIIIIIIIIIIIIIIIIYYII', 'IIIIIIIIIIIIIIIIIIZZII', 'IIIIIIIIIIIIIIIIXXIIII', 'IIIIIIIIIIIIIIIIYYIIII', 'IIIIIIIIIIIIIIIIZZIIII', 'IIIIIIIIIIIIIIXXIIIIII', 'IIIIIIIIIIIIIIYYIIIIII', 'IIIIIIIIIIIIIIZZIIIIII', 'IIIIIIIIIIIIXXIIIIIIII', 'IIIIIIIIIIIIYYIIIIIIII', 'IIIIIIIIIIIIZZIIIIIIII', 'IIIIIIIIIIXXIIIIIIIIII', 'IIIIIIIIIIYYIIIIIIIIII', 'IIIIIIIIIIZZIIIIIIIIII', 'IIIIIIIIXXIIIIIIIIIIII', 'IIIIIIIIYYIIIIIIIIIIII', 'IIIIIIIIZZIIIIIIIIIIII', 'IIIIIIXXIIIIIIIIIIIIII', 'IIIIIIYYIIIIIIIIIIIIII', 'IIIIIIZZIIIIIIIIIIIIII', 'IIIIXXIIIIIIIIIIIIIIII', 'IIIIYYIIIIIIIIIIIIIIII', 'IIIIZZIIIIIIIIIIIIIIII', 'IIXXIIIIIIIIIIIIIIIIII', 'IIYYIIIIIIIIIIIIIIIIII', 'IIZZIIIIIIIIIIIIIIIIII', 'XXIIIIIIIIIIIIIIIIIIII', 'YYIIIIIIIIIIIIIIIIIIII', 'ZZIIIIIIIIIIIIIIIIIIII', 'XIIIIIIIIIIIIIIIIIIIIX', 'YIIIIIIIIIIIIIIIIIIIIY', 'ZIIIIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIIIXXI', 'IIIIIIIIIIIIIIIIIIIYYI

In [4]:
import numpy as np

bitstring_matrix = np.array([[i % 2 == 0 for i in range(num_spins)]], dtype=bool)
bitstring_matrix = np.concatenate((bitstring_matrix, np.roll(bitstring_matrix, 1, axis=1)))
for i in range(num_spins):
    for j in range(num_spins // 2):
        domain_wall = bitstring_matrix[0].copy()
        domain_wall[i] -= 1
        domain_wall[(i + 1 + j * 2) % num_spins] -= 1
        bitstring_matrix = np.concatenate((bitstring_matrix, np.expand_dims(domain_wall, axis=0)))
        domain_wall = bitstring_matrix[1].copy()
        domain_wall[i] -= 1
        domain_wall[(i + 1 + j * 2) % num_spins] -= 1
        bitstring_matrix = np.concatenate((bitstring_matrix, np.expand_dims(domain_wall, axis=0)))

print(bitstring_matrix)

[[ True False  True ... False  True False]
 [False  True False ...  True False  True]
 [False  True  True ... False  True False]
 ...
 [False  True False ...  True False False]
 [ True False  True ... False False  True]
 [False  True False ...  True  True False]]


Next the results can be post-processed using the `solve_qubit()` method

In [6]:
from qiskit_addon_sqd.qubit import solve_qubit

scipy_kwargs = {"k": 4, "which": "SA"}
eigenvalues, eigenstates = solve_qubit(bitstring_matrix, xxz_hamiltonian, verbose=True, **scipy_kwargs)

min_eval = eigenstates[:, 0]

Projecting term 1 out of 66: (0.3+0j) * IIIIIIIIIIIIIIIIIIIIXX ...
Projecting term 2 out of 66: (0.3+0j) * IIIIIIIIIIIIIIIIIIIIYY ...
Projecting term 3 out of 66: (1+0j) * IIIIIIIIIIIIIIIIIIIIZZ ...
Projecting term 4 out of 66: (0.3+0j) * IIIIIIIIIIIIIIIIIIXXII ...
Projecting term 5 out of 66: (0.3+0j) * IIIIIIIIIIIIIIIIIIYYII ...
Projecting term 6 out of 66: (1+0j) * IIIIIIIIIIIIIIIIIIZZII ...
Projecting term 7 out of 66: (0.3+0j) * IIIIIIIIIIIIIIIIXXIIII ...
Projecting term 8 out of 66: (0.3+0j) * IIIIIIIIIIIIIIIIYYIIII ...
Projecting term 9 out of 66: (1+0j) * IIIIIIIIIIIIIIIIZZIIII ...
Projecting term 10 out of 66: (0.3+0j) * IIIIIIIIIIIIIIXXIIIIII ...
Projecting term 11 out of 66: (0.3+0j) * IIIIIIIIIIIIIIYYIIIIII ...
Projecting term 12 out of 66: (1+0j) * IIIIIIIIIIIIIIZZIIIIII ...
Projecting term 13 out of 66: (0.3+0j) * IIIIIIIIIIIIXXIIIIIIII ...
Projecting term 14 out of 66: (0.3+0j) * IIIIIIIIIIIIYYIIIIIIII ...
Projecting term 15 out of 66: (1+0j) * IIIIIIIIIIIIZZIIIIIIII ...